# RubyPyMill Example: Ruby vs Python Radar Chart

This notebook is designed to be executed by **RubyPyMill / Papermill**.

- It defines parameters in the `parameters` cell.
- It generates a demo CSV from an in-notebook table (`generate_data`).
- It renders a radar chart to a PNG file (`graph_output`).
- Optionally displays the image (`graph_view`).

## Tag layout (for RubyPyMill)

- `parameters`
- `setup`
- `generate_data`
- `graph_view`
- `graph_output`

### Recommended execution tag patterns

- cell_tags:"parameters,setup,generate_data,graph_view,graph_output"  
  Run all cells (demo / first-time execution)

- cell_tags: "parameters,setup,graph_view"  
  Preview the chart only (display only, no side effects)

- cell_tags: "parameters,setup,graph_output"  
  Generate the chart image (PNG output)


## parameters
実行パラメータ。未指定でもそのまま動作します。

これらの値は、外部の JSON パラメータによって上書き可能です。



In [ ]:
# parameters

csv_path = "examples/data/lang_skills.csv"
output_png = "examples/outputs/lang_radar.png"
title = "Ruby vs Python: Strength Profile (1–5)"

## setup
描画用関数の定義（副作用なし）。

このセルは副作用を持たず、以降のセルから再利用される関数のみを定義します。

The radar chart starts at 12 o’clock for better readability.

In [ ]:
# setup

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

def load_scores(csv_path):
    return pd.read_csv(csv_path)

def plot_radar(df, title):
    labels = df["dimension"].unique()
    angles = np.linspace(0, 2 * np.pi, len(labels), endpoint=False).tolist()
    angles += angles[:1]

    fig, ax = plt.subplots(figsize=(6, 6), subplot_kw=dict(polar=True))

    # ---- polar axis setup (coordinate system) ----
    ax.set_theta_offset(np.pi / 2)    # 12 o'clock start
    ax.set_theta_direction(-1)        # clockwise
    ax.set_xticks(angles[:-1])
    ax.set_xticklabels(labels)
    ax.set_ylim(0, 5)
    # ---- plot data ----
    for lang in df["language"].unique():
        values = df[df["language"] == lang]["score"].tolist()
        values += values[:1]
        ax.plot(angles, values, label=lang)
        ax.fill(angles, values, alpha=0.25)

# ---- decorations ----
    ax.set_title(title)
    ax.legend(loc="upper right")
    return fig

## generate_data
Ruby と Python の得意分野を 1–5 の相対スコアで定義し CSV を生成します。

このセルでは、Ruby と Python の「得意分野の傾向」を  
1〜5 の相対スコアとして定義しています。

ここでの数値は、言語の優劣を示すものではなく、  
**PoC や業務自動化における役割分担のイメージ**を表したものです。

- Ruby は DSL・制御・オーケストレーション寄り
- Python は データ処理・可視化・Notebook 実行寄り

数値はサンプルであり、自由に変更して試すことができます。

> Note: These scores are illustrative and represent typical usage tendencies, not language superiority.

In [ ]:
# generate_data

import os
import pandas as pd

data = [
    ("Ruby", "DSL/Orchestration", 5),
    ("Ruby", "Data Wrangling", 4),
    ("Ruby", "Notebook Workflow", 2),
    ("Ruby", "Visualization", 3),
    ("Ruby", "Packaging/Deployment", 5),
    ("Python", "DSL/Orchestration", 3),
    ("Python", "Data Wrangling", 5),
    ("Python", "Notebook Workflow", 5),
    ("Python", "Visualization", 5),
    ("Python", "Packaging/Deployment", 4),
]

df = pd.DataFrame(data, columns=["language", "dimension", "score"])
os.makedirs(os.path.dirname(csv_path), exist_ok=True)
df.to_csv(csv_path, index=False)

print(f"generated: {csv_path}")
df.head()

## graph_view
生成済み CSV を読み込み、その場で表示します（保存なし）。

In [ ]:
# graph_view

df = load_scores(csv_path)
fig = plot_radar(df, title)
plt.show()

## graph_output
生成済み CSV からグラフを作成し PNG として保存します。

In [ ]:
# graph_output

df = load_scores(csv_path)
fig = plot_radar(df, title)

import os
os.makedirs(os.path.dirname(output_png), exist_ok=True)
fig.savefig(output_png)
print(f"saved: {output_png}")